<a href="https://colab.research.google.com/github/21amY26/employee-salary-prediction/blob/main/01_empl_sal_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving employee_salary_dataset.csv to employee_salary_dataset.csv


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('employee_salary_dataset.csv')
df.head()

,Age,Education Level,Job Title,Experience (Years),Industry,Hours/Week,Work Mode,Skills,Salary (USD)
0,32,Bachelor's,Machine Learning Engineer,3,Manufacturing,49,Onsite,"Deep Learning, Python, ML",96189
1,36,Master's,Research Scientist,13,Utilities,49,Remote,"R, Python, Data Analysis",110513
2,28,Bachelor's,Financial Analyst,7,Healthcare,36,Remote,"Valuation, Python",80428
3,47,Bachelor's,Technician,9,Manufacturing,43,Onsite,"Safety, CNC, Maintenance",52344
4,57,Master's,Accountant,12,IT,41,Onsite,"Tally, Compliance",72856


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Age                 200 non-null    int64 
 1   Education Level     200 non-null    object
 2   Job Title           200 non-null    object
 3   Experience (Years)  200 non-null    int64 
 4   Industry            200 non-null    object
 5   Hours/Week          200 non-null    int64 
 6   Work Mode           200 non-null    object
 7   Skills              200 non-null    object
 8   Salary (USD)        200 non-null    int64 
dtypes: int64(4), object(5)
memory usage: 14.2+ KB


In [6]:
df.describe(include='all')

,Age,Education Level,Job Title,Experience (Years),Industry,Hours/Week,Work Mode,Skills,Salary (USD)
count,200.0000,200,200,200.000000,200,200.000000,200,200,200.000000
unique,NaN,5,20,NaN,11,NaN,3,127,NaN
top,NaN,Bachelor's,Marketing Manager,NaN,Insurance,NaN,Onsite,"CRM, Communication",NaN
freq,NaN,70,18,NaN,24,NaN,108,8,NaN
mean,40.7850,NaN,NaN,9.905000,NaN,42.210000,NaN,NaN,84511.600000
std,11.3021,NaN,NaN,8.610848,NaN,4.560691,NaN,NaN,24259.191906
min,22.0000,NaN,NaN,0.000000,NaN,35.000000,NaN,NaN,39233.000000
25%,30.0000,NaN,NaN,3.000000,NaN,38.000000,NaN,NaN,67611.500000
50%,41.0000,NaN,NaN,8.000000,NaN,42.000000,NaN,NaN,80073.000000
75%,50.0000,NaN,NaN,14.250000,NaN,46.000000,NaN,NaN,100411.500000


In [7]:
df.isnull().sum()

,0
Age,0
Education Level,0
Job Title,0
Experience (Years),0
Industry,0
Hours/Week,0
Work Mode,0
Skills,0
Salary (USD),0


In [8]:
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer, StandardScaler
le=LabelEncoder()
mlb=MultiLabelBinarizer()
scaler=StandardScaler()

In [9]:
#label encoding for education levels
'''
df['Education Level Encoded'] = le.fit_transform(df['Education Level'])
dict(zip(le.classes_,le.transform(le.classes_)))

output given :
{"Bachelor's": np.int64(0),
 'Diploma': np.int64(1),
 'High School': np.int64(2),
 "Master's": np.int64(3),
 'PhD': np.int64(4)}
'''

education_order={
     'High School':0,
     'Diploma':1,
     'Bachelor\'s':2,
     'Master\'s':3,
     'PhD':4
 }

df['Education Level Encoded']=df['Education Level'].map(education_order)
df.drop('Education Level',axis=1,inplace=True)

In [10]:
#one hot encoding for work mode : hybrid/onsite/remote
df=pd.get_dummies(df,columns=['Work Mode'])

In [11]:
#one- hot encoding for job title
df=pd.get_dummies(df,columns=['Job Title'])

In [14]:
#one hot encoding for industry
df=pd.get_dummies(df,columns=['Industry'])

In [12]:
# multilabel for Skills, since it has multiple comma separated values
skills_split=df['Skills'].str.split(', ')
skills_encoded=pd.DataFrame(mlb.fit_transform(skills_split),columns=mlb.classes_)

#back to main DF
df=pd.concat([df,skills_encoded],axis=1)
df.drop('Skills',axis=1,inplace=True)


In [13]:
#normalising numerical data (age, exp, hrs/wk)

cols_to_scale=['Age','Experience (Years)', 'Hours/Week']
df[cols_to_scale]=scaler.fit_transform(df[cols_to_scale])


In [15]:
df.columns

Index(['Age', 'Experience (Years)', 'Hours/Week', 'Salary (USD)',
       'Education Level Encoded', 'Work Mode_Hybrid', 'Work Mode_Onsite',
       'Work Mode_Remote', 'Job Title_Accountant',
       'Job Title_Business Analyst', 'Job Title_Cloud Engineer',
       'Job Title_Content Writer', 'Job Title_Data Scientist',
       'Job Title_DevOps Engineer', 'Job Title_Electrician',
       'Job Title_Financial Analyst', 'Job Title_Graphic Designer',
       'Job Title_HR Manager', 'Job Title_Machine Learning Engineer',
       'Job Title_Marketing Manager', 'Job Title_Product Manager',
       'Job Title_Project Manager', 'Job Title_QA Engineer',
       'Job Title_Research Scientist', 'Job Title_Sales Executive',
       'Job Title_Software Engineer', 'Job Title_Technician',
       'Job Title_UI/UX Designer', 'AWS', 'Agile', 'Analytics', 'Azure',
       'CI/CD', 'CNC', 'CRM', 'Communication', 'Compliance', 'Data Analysis',
       'Deep Learning', 'Docker', 'Excel', 'Figma', 'Git', 'HRM', 'HTML/C

In [18]:
#checking for zeroes
df.isnull().sum().sort_values(ascending=False)

,0
Age,0
Experience (Years),0
Hours/Week,0
Salary (USD),0
Education Level Encoded,0
...,...
Industry_Manufacturing,0
Industry_Media,0
Industry_Retail,0
Industry_Tech,0


In [19]:
# saving preprocessed csv
df.to_csv('prep_emp_sal_data.csv',index='False')
files.download('prep_emp_sal_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>